# server connect

In [32]:
!pip install anvil-uplink
import anvil.server

anvil.server.connect("server_2BWTWXDY2V4ON7JLQ4HILKDO-QMTVJN4LDJALI4EQ")



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)
Reconnecting Anvil Uplink...
Connecting to wss://anvil.works/uplink
Anvil websocket open
Fatal error from Anvil server: Incorrect uplink key
Anvil websocket closed (code 1000, reason=b'')


Disconnecting from previous connection first...
Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Debug for azrafe7@gmail.com" as SERVER


# start

In [2]:
# constansts

START_WORD_LENGTHS = [4,5,6]

MIN_WORD_LENGTH = 3
MAX_WORD_LENGTH = 9

NUM_STEPS = 8
MIN_STEPS = 5


In [3]:
'''
Created on Feb 7, 2013

@author: olegs
'''

ROMAN_CONSTANTS = (
            ( "", "I", "II", "III", "IV", "V", "VI", "VII", "VIII", "IX" ),
            ( "", "X", "XX", "XXX", "XL", "L", "LX", "LXX", "LXXX", "XC" ),
            ( "", "C", "CC", "CCC", "CD", "D", "DC", "DCC", "DCCC", "CM" ),
            ( "", "M", "MM", "MMM", "",   "",  "-",  "",    "",     ""   ),
        )

ROMAN_SYMBOL_MAP = dict(I=1, V=5, X=10, L=50, C=100, D=500, M=1000)

CUTOFF = 4000
BIG_DEC = 2900
BIG_ROMAN = "MMCM"
ROMAN_NOUGHT = "nulla"

def digits(num):
    if num < 0:
        raise Exception('range error: negative numbers not supported')
    if num % 1 != 0.0:
        raise Exception('floating point numbers not supported')
    res = []
    while num > 0:
        res.append(num % 10)
        num //= 10
    return res

def toString(num, emptyZero=False):
    if num < CUTOFF:
        digitlist = digits(num)
        if digitlist:
            res = reversed([ ROMAN_CONSTANTS[order][digit] for order, digit in enumerate(digitlist) ])
            return "".join(res)
        else:
            return "" if emptyZero else ROMAN_NOUGHT 
    else:
        if num % 1 != 0.0:
            raise Exception('floating point numbers not supported')
        # For numbers over or equal the CUTOFF, the remainder of division by 2900
        # is represented as above, prepended with the multiples of MMCM (2900 in Roman),
        # which guarantees no more than 3 repetitive Ms.
        return BIG_ROMAN * (num // BIG_DEC) + toString(num % BIG_DEC, emptyZero=True)

def parse(numeral):
    numeral = numeral.upper()
    result = 0

    lastVal = 0
    lastCount = 0
    subtraction = False
    for symbol in numeral[::-1]:
        value = ROMAN_SYMBOL_MAP.get(symbol)
        if not value:
            return False
        if lastVal == 0:
            lastCount = 1
            lastVal = value
        elif lastVal == value:
            lastCount += 1
            # exceptions
        else:
            result += (-1 if subtraction else 1) * lastVal * lastCount
            subtraction = lastVal > value
            lastCount = 1
            lastVal = value
    
    result = result + (-1 if subtraction else 1) * lastVal * lastCount
    return True

# loadDict

In [4]:
import requests
import time

dictionaryFileName = 'assets/dictionary_FULL_IT.txt'
dictionaryFileName = 'assets/Ita.txt'

    
def normalizeWord(word):
    return word.strip().upper()
    
LOAD_FROM_URL = True

t0 = time.time()

lines = []
if LOAD_FROM_URL:
    baseUrl = 'https://azrafe7.github.io/'
    f = requests.get(baseUrl + dictionaryFileName)
    lines = f.text.splitlines()
else:
    with open(dictionaryFileName) as f:
        lines = f.read().splitlines()

filteredWords = []
for line in lines:
    word = normalizeWord(line)
    length = len(word)
    if length >= MIN_WORD_LENGTH and length <= MAX_WORD_LENGTH and not parse(word):
        filteredWords.append(word)

print("Dictionary loaded in {0:.2f}s".format(time.time() - t0))

print('{0}/{1}'.format(len(filteredWords), len(lines)))

print(filteredWords[:10])

Dictionary loaded in 1.46s
401123/710993
['AALENIANA', 'AALENIANE', 'AALENIANI', 'AALENIANO', 'ABABUA', 'ABACA', 'ABACETI', 'ABACETO', 'ABACHI', 'ABACHISTA']


In [5]:

def wordsOfLength(lengths):
    return [word for word in filteredWords if len(word) in lengths]

len(list(wordsOfLength(START_WORD_LENGTHS)))


35094

In [6]:

def isAnagram(sourceWord, targetWord):
    if len(sourceWord) != len(targetWord):
        return False
    
    res = True
    
    sourceCharSet = list(sourceWord)
    targetCharSet = list(targetWord)
    
    #print(targetCharSet)
    for i, ch in enumerate(sourceWord):
        #print(ch, targetCharSet)
        if ch in targetCharSet:
            targetCharSet.remove(ch)
        else:
            res = False
            break
    
    return res
    
        
assert(isAnagram('', ''))
assert(not isAnagram('corto', 'trOca'))
assert(not isAnagram('c', ''))
assert(isAnagram('cortao', 'trocao'))

srcWord = 'ostra'
dstWord ='motri'

print('isAnagram("{0}", "{1}")'.format(srcWord, dstWord))
print(isAnagram(srcWord, dstWord))

isAnagram("ostra", "motri")
False


In [7]:
import time

def isAnagramEx(sourceWord, targetWord, maxDiffLength=0, strict=False):
    if maxDiffLength == 0:
        return isAnagram(sourceWord, targetWord)
    else:
        wordDiffLength = abs(len(sourceWord) - len(targetWord))
        if wordDiffLength > maxDiffLength:
            return False
        
    sourceCharSet = list(sourceWord)
    targetCharSet = list(targetWord)
    
    if len(sourceCharSet) > len(targetCharSet):
        sourceCharSet, targetCharSet = targetCharSet, sourceCharSet
        
    #print(targetCharSet, sourceCharSet)
    for ch in sourceCharSet:
        #print(ch, targetCharSet)
        if ch in targetCharSet:
            targetCharSet.remove(ch)
    
    #print(targetCharSet)
    targetCharSetLength = len(targetCharSet)
    #print(wordDiffLength, targetCharSet)
    isLooseAnagram = (targetCharSetLength <= maxDiffLength)
    res = (not strict and isLooseAnagram) or (strict and isLooseAnagram and wordDiffLength == maxDiffLength)
    return res
    
        
maxDiffLength=2

assert(isAnagramEx('', ''))
assert(not isAnagramEx('corto', 'trOca'))
assert(not isAnagramEx('c', ''))
assert(isAnagramEx('cortao', 'trocao'))
assert(isAnagramEx('cortao', 'trocaox', 1))
assert(not isAnagramEx('strappo', 'posta', 1))
assert(isAnagramEx('strappo', 'posta', 2))
assert(isAnagramEx('strappo', 'posta', 3))
assert(not isAnagramEx('strappo', 'posta', 3, strict=True))
assert(isAnagramEx('strappo', 'posta', 2, strict=True))
assert(isAnagramEx('LEVI', 'ABILE', 2,))
assert(not isAnagramEx('TUFFAI', 'ABACA', 2,))


srcWord = 'straccio'
dstWord ='crosta'

print('isAnagramEx("{0}", "{1}", {2})'.format(srcWord, dstWord, maxDiffLength))
print(isAnagramEx(srcWord, dstWord, maxDiffLength=maxDiffLength))

t0 = time.time()
for i in range(0):
  isAnagramEx('', '')
  not isAnagramEx('corto', 'trOca')
  not isAnagramEx('c', '')
  isAnagramEx('cortao', 'trocao')
  isAnagramEx('cortao', 'trocaox', 1)
  not isAnagramEx('strappo', 'posta', 1)
  isAnagramEx('strappo', 'posta', 2)
  isAnagramEx('strappo', 'posta', 3)
  not isAnagramEx('strappo', 'posta', 3, strict=True)
  isAnagramEx('strappo', 'posta', 2, strict=True)
  isAnagramEx('LEVI', 'ABILE', 2, )
  not isAnagramEx('TUFFAI', 'ABACA', 2, )  
print(f'{(time.time() - t0):.4f}s')


isAnagramEx("straccio", "crosta", 2)
True
0.0003s


# isAnagramEx2

In [8]:
import time


def _isAnagramEx2(sortedSourceChars, sortedTargetChars, sourceLength, targetLength, maxDiffLength=0, strict=False, failFast=False):
  if failFast and abs(sourceLength - targetLength) > maxDiffLength:
    return False, -1

  # if maxDiffLength == 0:
  #     return sortedSourceChars == sortedTargetChars
  # else:
  #   wordDiffLength = abs(sourceLength - targetLength)
  #   if wordDiffLength > maxDiffLength:
  #       return False

  i = 0
  j = 0
  diff = 0
  same = 0
  if sourceLength != 0 and targetLength != 0:
    while i < sourceLength and j < targetLength:
      sourceCh = sortedSourceChars[i]
      targetCh = sortedTargetChars[j]
      if sourceCh == targetCh:
        i += 1
        j += 1
        same += 1
      else:
        if sourceCh < targetCh:
          i += 1
        else:
          j += 1

  diff = max(sourceLength, targetLength) - same
  #print(sortedSourceChars, sortedTargetChars, diff, same)
  isLooseAnagram = diff <= maxDiffLength
  res = (not strict and isLooseAnagram) or (strict and isLooseAnagram and diff == maxDiffLength)
  return res, diff


def isAnagramEx2(sourceWord, targetWord, maxDiffLength=0, strict=False, failFast=False):
  sortedSourceChars = sorted(sourceWord)
  sortedTargetChars = sorted(targetWord)

  # if maxDiffLength == 0:
  #     return sortedSourceChars == sortedTargetChars
  # else:


  sourceLength = len(sortedSourceChars)
  targetLength = len(sortedTargetChars)

  return _isAnagramEx2(sortedSourceChars, sortedTargetChars, sourceLength, targetLength, maxDiffLength=maxDiffLength, strict=strict, failFast=failFast)

maxDiffLength = 2

assert (True, 0) == isAnagramEx2('', '')
assert (False, 2) == isAnagramEx2('corto', 'trOca')
assert (False, 4) == isAnagramEx2('corto', 'trOcaul')
assert (False, 1) == isAnagramEx2('c', '')
assert (True, 1) == isAnagramEx2('cortao', 'trocaox', 1)
assert (True, 0) == isAnagramEx2('cortao', 'trocao')
assert (False, 2) == isAnagramEx2('strappo', 'posta', 1)
assert (True, 2) == isAnagramEx2('strappo', 'posta', 2)
assert (True, 2) == isAnagramEx2('strappo', 'posta', 3)
assert (False, 2) == isAnagramEx2('strappo', 'posta', 3, strict=True)
assert (True, 2) == isAnagramEx2('strappo', 'posta', 2, strict=True)
assert (True, 2) == isAnagramEx2('LEVI', 'ABILE', 2, strict=False)
assert (False, 5) == isAnagramEx2('TUFFAI', 'ABACA', 2, strict=False)
assert (False, 5) == isAnagramEx2('TUFFAI', 'ABACA', 2, strict=True)
assert (True, 1) == isAnagramEx2('AOPST', 'AOPSTU', 1, strict=False)
assert (True, 1) == isAnagramEx2('AOPST', 'AOPSTU', 1, strict=True)

srcWord = 'straccio'
dstWord = 'crosta'

print('isAnagramEx2("{0}", "{1}", {2})'.format(srcWord, dstWord, maxDiffLength))
print(isAnagramEx2(srcWord, dstWord, maxDiffLength=maxDiffLength))

t0 = time.time()
for i in range(0):
  isAnagramEx2('', '')
  not isAnagramEx2('corto', 'trOca')
  not isAnagramEx2('c', '')
  isAnagramEx2('cortao', 'trocao')
  isAnagramEx2('cortao', 'trocaox', 1)
  not isAnagramEx2('strappo', 'posta', 1)
  isAnagramEx2('strappo', 'posta', 2)
  isAnagramEx2('strappo', 'posta', 3)
  not isAnagramEx2('strappo', 'posta', 3, strict=True)
  isAnagramEx2('strappo', 'posta', 2, strict=True)
  isAnagramEx2('LEVI', 'ABILE', 2, )
  not isAnagramEx2('TUFFAI', 'ABACA', 2, )
print(f'{(time.time() - t0):.4f}s')


isAnagramEx2("straccio", "crosta", 2)
(True, 2)
0.0002s


In [9]:
import time
from collections import Counter

def isAnagram_Counter(sourceWord, targetWord, maxDiffLength=0, strict=False):
  sourceCounter = Counter(sourceWord)
  targetCounter = Counter(targetWord)

  # swap? (lengthier first!)
  if len(sourceWord) < len(targetWord):
    sourceCounter, targetCounter = targetCounter, sourceCounter

  diffCounter = sourceCounter - targetCounter

  diffSum = sum(diffCounter.values())

  isAnagram = (not strict and diffSum <= maxDiffLength) or (strict and diffSum == maxDiffLength)
  return isAnagram, diffSum

print("->", isAnagram_Counter('corto', 'trOca'))


assert (True, 0) == isAnagram_Counter('', '')
assert (False, 2) == isAnagram_Counter('corto', 'trOca')
assert (False, 4) == isAnagram_Counter('corto', 'trOcaul')
assert (False, 1) == isAnagram_Counter('c', '')
assert (True, 1) == isAnagram_Counter('cortao', 'trocaox', 1)
assert (True, 0) == isAnagram_Counter('cortao', 'trocao')
assert (False, 2) == isAnagram_Counter('strappo', 'posta', 1)
assert (True, 2) == isAnagram_Counter('strappo', 'posta', 2)
assert (True, 2) == isAnagram_Counter('strappo', 'posta', 3)
assert (False, 2) == isAnagram_Counter('strappo', 'posta', 3, strict=True)
assert (True, 2) == isAnagram_Counter('strappo', 'posta', 2, strict=True)
assert (True, 2) == isAnagram_Counter('LEVI', 'ABILE', 2, strict=False)
assert (False, 5) == isAnagram_Counter('TUFFAI', 'ABACA', 2, strict=False)
assert (False, 5) == isAnagram_Counter('TUFFAI', 'ABACA', 2, strict=True)
assert (True, 1) == isAnagram_Counter('AOPST', 'AOPSTU', 1, strict=False)
assert (True, 1) == isAnagram_Counter('AOPST', 'AOPSTU', 1, strict=True)
assert (True, 1) == isAnagram_Counter('BAVA', 'ABAVI', 1, strict=True)

t0 = time.time()
for i in range(0):
  isAnagram_Counter('', '')
  not isAnagram_Counter('corto', 'trOca')
  not isAnagram_Counter('c', '')
  isAnagram_Counter('cortao', 'trocao')
  isAnagram_Counter('cortao', 'trocaox', 1)
  not isAnagram_Counter('strappo', 'posta', 1)
  isAnagram_Counter('strappo', 'posta', 2)
  isAnagram_Counter('strappo', 'posta', 3)
  not isAnagram_Counter('strappo', 'posta', 3, strict=True)
  isAnagram_Counter('strappo', 'posta', 2, strict=True)
  isAnagram_Counter('LEVI', 'ABILE', 2, )
  not isAnagram_Counter('TUFFAI', 'ABACA', 2, )
print(f'{(time.time() - t0):.4f}s')


-> (False, 2)
0.0002s


In [10]:
import time
from copy import copy

class Counter2(dict):
  
  def __init__(self, word):
    self.word = word
    for ch in word:
      if ch not in self:
        self[ch] = 0
      self[ch] += 1
      # if ch not in self:
      #   self[ch] = word.count(ch)

  def __add__(self, otherCounter):
    newCounter = copy(self)
    for k, v in otherCounter.items():
      if k not in newCounter:
        newCounter[k] = 0
      newCounter[k] += v
    
    return newCounter

  def __sub__(self, otherCounter):
    newCounter = copy(self)
    for k, v in otherCounter.items():
      if k not in newCounter:
        newCounter[k] = 0
      else:
        newCounter[k] -= v
        if newCounter[k] <= 0:
          del newCounter[k]
    
    return newCounter

  def isAnagram(self, otherCounter, maxDiffLength=1, strict=False, failFast=False):
    sourceCounter, targetCounter = self, otherCounter
    sourceLength, targetLength = len(sourceCounter.word), len(targetCounter.word)
    if failFast and abs(targetLength - sourceLength) > maxDiffLength:
      return False, -1
    if sourceLength < targetLength:
      sourceCounter, targetCounter = targetCounter, sourceCounter
      sourceLength, targetLength = targetLength, sourceLength
    diffCounter = sourceCounter - targetCounter
    diffSum = sum(diffCounter.values())
   
    isAnagram = (not strict and diffSum <= maxDiffLength) or (strict and diffSum == maxDiffLength)
    return isAnagram, diffSum


def isAnagram_Counter2(sourceWord, targetWord, maxDiffLength=0, strict=False):
  sourceCounter = Counter2(sourceWord)
  targetCounter = Counter2(targetWord)

  # swap? (lengthier first!)
  if len(sourceWord) < len(targetWord):
    sourceCounter, targetCounter = targetCounter, sourceCounter

  diffCounter = sourceCounter - targetCounter

  diffSum = sum(diffCounter.values())

  isAnagram = (not strict and diffSum <= maxDiffLength) or (strict and diffSum == maxDiffLength)
  return isAnagram, diffSum

print("->", isAnagram_Counter2('corto', 'trOca'))

word1 = Counter2('corto')
word2 = Counter2('trOca')
display(word1)
display(word2)
display(word1 + word2)
display(word1 - word2)
display(Counter('corto') - Counter('trOca'))

assert (True, 0) == isAnagram_Counter2('', '')
assert (False, 2) == isAnagram_Counter2('corto', 'trOca')
assert (False, 4) == isAnagram_Counter2('corto', 'trOcaul')
assert (False, 1) == isAnagram_Counter2('c', '')
assert (True, 1) == isAnagram_Counter2('cortao', 'trocaox', 1)
assert (True, 0) == isAnagram_Counter2('cortao', 'trocao')
assert (False, 2) == isAnagram_Counter2('strappo', 'posta', 1)
assert (True, 2) == isAnagram_Counter2('strappo', 'posta', 2)
assert (True, 2) == isAnagram_Counter2('strappo', 'posta', 3)
assert (False, 2) == isAnagram_Counter2('strappo', 'posta', 3, strict=True)
assert (True, 2) == isAnagram_Counter2('strappo', 'posta', 2, strict=True)
assert (True, 2) == isAnagram_Counter2('LEVI', 'ABILE', 2, strict=False)
assert (False, 5) == isAnagram_Counter2('TUFFAI', 'ABACA', 2, strict=False)
assert (False, 5) == isAnagram_Counter2('TUFFAI', 'ABACA', 2, strict=True)
assert (True, 1) == isAnagram_Counter2('AOPST', 'AOPSTU', 1, strict=False)
assert (True, 1) == isAnagram_Counter2('AOPST', 'AOPSTU', 1, strict=True)
assert (True, 1) == isAnagram_Counter2('BAVA', 'ABAVI', 1, strict=True)
assert (True, 1) == Counter2('BAVA').isAnagram(Counter2('ABAVI'), maxDiffLength=1)

t0 = time.time()
for i in range(0):
  isAnagram_Counter2('', '')
  not isAnagram_Counter2('corto', 'trOca')
  not isAnagram_Counter2('c', '')
  isAnagram_Counter2('cortao', 'trocao')
  isAnagram_Counter2('cortao', 'trocaox', 1)
  not isAnagram_Counter2('strappo', 'posta', 1)
  isAnagram_Counter2('strappo', 'posta', 2)
  isAnagram_Counter2('strappo', 'posta', 3)
  not isAnagram_Counter2('strappo', 'posta', 3, strict=True)
  isAnagram_Counter2('strappo', 'posta', 2, strict=True)
  isAnagram_Counter2('LEVI', 'ABILE', 2, )
  not isAnagram_Counter2('TUFFAI', 'ABACA', 2, )
print(f'{(time.time() - t0):.4f}s')


-> (False, 2)


{'c': 1, 'o': 2, 'r': 1, 't': 1}

{'t': 1, 'r': 1, 'O': 1, 'c': 1, 'a': 1}

{'c': 2, 'o': 2, 'r': 2, 't': 2, 'O': 1, 'a': 1}

{'o': 2, 'O': 0, 'a': 0}

Counter({'o': 2})

0.0003s


# TreeNode

In [11]:
import random

def counterCompare(counterA, counterB):
  # counterA = Counter2(wordA)
  # counterB = Counter2(wordB)
  #isAnagram, diff = counterA.isAnagram(counterB, maxDiffLength=1, strict=True, failFast=True)
  isAnagram, diff = isAnagramEx2(counterA.word, counterB.word, maxDiffLength=1, strict=True)
  #print(counterA.word, counterB.word, (isAnagram, diff))
  return -1 if isAnagram else 1

def wordCompare(wordA, wordB):
  isAnagram, diff = isAnagramEx2(wordA, wordB, maxDiffLength=1, strict=True, failFast=True)
  #print(counterA.word, counterB.word, (isAnagram, diff))
  return -1 if isAnagram else 1

class TreeNode:
  def __init__(self, word):
    self.left = None
    self.right = None
    self.data = word

  def insert(self, word, compare=wordCompare):
    data = word
    
    y = None
    node = self
    i = 0
    while node is not None:
      i += 1
      if node.data:
        y = node
        comparison = compare(node.data, data)
        if comparison < 0:
          node = node.left
        else:
          node = node.right

    # if random.choice([0,1,2,4,5]) == 0:
    #   print(i)
    
    if y is None:
      self.data = data
    elif comparison < 0:
      y.left = TreeNode(word)
    else:
      y.right = TreeNode(word)

  def search(self, word, compare=wordCompare):
    results = []

    data = word
    
    y = None
    node = self
    while node is not None:
      if node.data:
        y = node
        comparison = compare(node.data, data)
        #print(comparison, node.data.word, data.word)
        if comparison < 0:
          results.append(node.data)
          node = node.left
        else:
          node = node.right

    return results


def printTree(node, level=0):
  if node != None:
    printTree(node.left, level + 1)
    print(' ' * 4 * level + '-> ' + str(node.word))
    printTree(node.right, level + 1)



t0 = time.time()
root = TreeNode('@')
shuffledWords = filteredWords[:]
random.shuffle(shuffledWords)
for i, word in enumerate(shuffledWords[:10000]):
  root.insert(word)
  if i % 1000 == 0:
    print(i, word)
print(f'build took {(time.time() - t0):.2f}')

#printTree(root)

t0 = time.time()
word = shuffledWords[1]
res = root.search(word)
print(f'search {word} took {(time.time() - t0):.2f}')

display(res)


0 AZIONAGLI
1000 INDORSATO
2000 ISTESSI
Reconnecting Anvil Uplink...
Connecting to wss://anvil.works/uplink
Anvil websocket open
Fatal error from Anvil server: Incorrect uplink key
Anvil websocket closed (code 1000, reason=b'')
Reconnecting Anvil Uplink...
Connecting to wss://anvil.works/uplink
Anvil websocket open
Fatal error from Anvil server: Incorrect uplink key
Anvil websocket closed (code 1000, reason=b'')
3000 DERIMENTE
4000 SBARACCAI
Reconnecting Anvil Uplink...
Connecting to wss://anvil.works/uplink
Anvil websocket open
Fatal error from Anvil server: Incorrect uplink key
Anvil websocket closed (code 1000, reason=b'')
5000 SLUMANTI
Reconnecting Anvil Uplink...
Connecting to wss://anvil.works/uplink
Anvil websocket open
Fatal error from Anvil server: Incorrect uplink key
Anvil websocket closed (code 1000, reason=b'')
6000 BRUCIOLI
Reconnecting Anvil Uplink...
Connecting to wss://anvil.works/uplink
Anvil websocket open
Fatal error from Anvil server: Incorrect uplink key
Anvil web

[]

In [12]:
#print(shuffledWords[:100])
t0 = time.time()
word = random.choice(shuffledWords[:5000])
res = root.search(word)
print(f'search {word} took {(time.time() - t0):.2f}')

display(res)


search SGRUMAVA took 0.02


[]

# Trie

In [13]:
# http://stevehanov.ca/blog/index.php?id=114
# By Steve Hanov, 2011. Released to the public domain
import time
import sys

DICTIONARY = "/usr/share/dict/words";
TARGET = "SCORTA"
MAX_COST = 4

# Keep some interesting statistics
NodeCount = 0
WordCount = 0

# The Trie data structure keeps a set of words, organized with one node for
# each letter. Each node has a branch for each letter that may follow it in the
# set of words.
class TrieNode:
    def __init__(self):
        self.word = None
        self.children = {}

        global NodeCount
        NodeCount += 1

    def insert( self, word ):
        node = self
        for letter in word:
            if letter not in node.children: 
                node.children[letter] = TrieNode()

            node = node.children[letter]

        node.word = word

# read dictionary file into a trie
start = time.time()

trie = TrieNode()
for word in filteredWords:
    WordCount += 1
    trie.insert( word )

end = time.time()
print("Build took %g s" % (end - start))

print("Read %d words into %d nodes" % (WordCount, NodeCount))

# The search function returns a list of all words that are less than the given
# maximum distance from the target word
def search( word, maxCost ):

    # build first row
    currentRow = range( len(word) + 1 )

    results = []

    # recursively search each branch of the trie
    for letter in trie.children:
        searchRecursive( trie.children[letter], letter, word, currentRow, 
            results, maxCost )

    return results

# This recursive helper is used by the search function above. It assumes that
# the previousRow has been filled in already.
def searchRecursive( node, letter, word, previousRow, results, maxCost ):

    columns = len( word ) + 1
    currentRow = [ previousRow[0] + 1 ]

    # Build one row for the letter, with a column for each letter in the target
    # word, plus one for the empty string at column 0
    for column in range( 1, columns ):

        insertCost = currentRow[column - 1] + 1
        deleteCost = previousRow[column] + 1

        if word[column - 1] != letter:
            replaceCost = previousRow[ column - 1 ] + 1
        else:                
            replaceCost = previousRow[ column - 1 ]

        currentRow.append( min( insertCost, deleteCost, replaceCost ) )

    # if the last entry in the row indicates the optimal cost is less than the
    # maximum cost, and there is a word in this trie node, then add it.
    if currentRow[-1] <= maxCost and node.word != None:
        results.append( (node.word, currentRow[-1] ) )

    # if any entries in the row are less than the maximum cost, then 
    # recursively search each branch of the trie
    if min( currentRow ) <= maxCost:
        for letter in node.children:
            searchRecursive( node.children[letter], letter, word, currentRow, 
                results, maxCost )

start = time.time()
results = search( TARGET, MAX_COST )
end = time.time()

#for result in results: print(result)

print("Search took %g s" % (end - start))



Reconnecting Anvil Uplink...
Connecting to wss://anvil.works/uplink
Build took 3.69534 s
Read 401123 words into 674739 nodes
Anvil websocket open
Fatal error from Anvil server: Incorrect uplink key
Anvil websocket closed (code 1000, reason=b'')
Search took 0.563971 s


In [14]:
import random

startWord = random.choice(filteredWords)
print("START WORD:", startWord)

start = time.time()
results = search(startWord, 3)
end = time.time()

print("Search took %g s" % (end - start))

validEndWord = random.choice(results)

print("END WORD:", validEndWord)


START WORD: CROSTARLA
Search took 0.238652 s
END WORD: ('PRESTARLO', 3)


findAnagramsEx

In [15]:
def findAnagramsEx(wordList, word, maxDiffLength=1, strict=False):
    results = []
    wordLength = len(word)
    for entry in wordList:
        entryLength = len(entry)
        if word == entry or entryLength < wordLength - maxDiffLength or entryLength > wordLength + maxDiffLength:
            continue
        if isAnagramEx(entry, word, maxDiffLength=maxDiffLength, strict=strict):
            diffLength = abs(len(entry) - len(word))
            results.append((entry, diffLength))
    
    return results


In [16]:
import random

newStartWord = random.choice(filteredWords)
print("START WORD:", startWord)

anagramsDict = {0:[], 1:[], 2:[]}
randomFilteredWords = filteredWords[:]
random.shuffle(filteredWords[:])
randomFilteredWords = sorted(randomFilteredWords)

anagrams = findAnagramsEx(randomFilteredWords, newStartWord, maxDiffLength=1, strict=False)
for anagram in anagrams:
    anagramsDict[anagram[1]].append(anagram[0])
    
print(anagrams)
#print(anagramsDict[0][:10])

START WORD: CROSTARLA
[('ADONTAVA', 0), ('AITAVANO', 0), ('ANATRANO', 0), ('ANATRAVO', 0), ('ANATRONA', 0), ('ANDAVANO', 0), ('ANFANATO', 0), ('ANFANAVO', 0), ('ANNASATO', 0), ('ANNASAVO', 0), ('ANNODATA', 0), ('ANNODAVA', 0), ('ANNOIATA', 0), ('ANNOIAVA', 0), ('ANNOMATA', 0), ('ANNOMAVA', 0), ('ANNOTALA', 0), ('ANNOTATA', 0), ('ANNOTAVA', 0), ('ANNOTAVI', 0), ('ANNOTAVO', 0), ('ANNOTTAVA', 1), ('ANSAVANO', 0), ('ARTAVANO', 0), ('ATTAVANO', 0), ('AVANZANO', 0), ('AVANZATO', 0), ('CANTAVANO', 1), ('CANTONAVA', 1), ('DATAVANO', 0), ('FATAVANO', 0), ('INNOVATA', 0), ('INTANAVA', 0), ('INTANAVO', 0), ('INTONAVA', 0), ('LONTANAVA', 1), ('MANTOVANA', 1), ('NASAVANO', 0), ('NATAVAMO', 0), ('NOTAVANO', 0), ('NOVANTA', 1), ('NOVATANE', 0), ('OSANNATA', 0), ('OSANNAVA', 0), ('PANAVANO', 0), ('PAVONATA', 0), ('RAVANANO', 0), ('RAVANATO', 0), ('SANAVANO', 0), ('STANAVANO', 1), ('TAFANANO', 0), ('TAFANAVO', 0), ('TANAVAMO', 0), ('TANAVANO', 0), ('TANCAVANO', 1), ('TANFANAVO', 1), ('TANFAVANO', 1), 

In [17]:
print(anagramsDict[1])

['ANNOTTAVA', 'CANTAVANO', 'CANTONAVA', 'LONTANAVA', 'MANTOVANA', 'NOVANTA', 'STANAVANO', 'TANCAVANO', 'TANFANAVO', 'TANFAVANO', 'TANGONAVA', 'TANNAVA', 'TANNAVAMO', 'TANNAVANO', 'TANNAVO', 'TRANAVANO', 'VANTANO', 'VANTAVANO']


# generateChain

In [36]:
import time

def generateChain(quiet=False, startWord=None, strict=False):
    startWords = wordsOfLength(START_WORD_LENGTHS)
    
    restart = True
    
    while restart:
        restart = False
        startWord = random.choice(startWords)
        print("START WORD:", startWord)
    
        steps = [startWord]
    
        prevStep = (startWord, 0)
        #self.info.text = ""
        for i in range(6):
            choice = random.choice(["anagram", "levenshtein"])
            if choice == "anagram":
                if not quiet: print("Finding anagrams...", end="")
                startTime = time.time()
                anagrams = findAnagramsEx(filteredWords, prevStep[0], maxDiffLength=1, strict=strict)
                if not quiet: print(" ({0:.2f})".format(time.time() - startTime))
                if len(anagrams) < 10 and i < MIN_STEPS:
                    restart = True
                    print(choice, len(anagrams))
                    break
            else:
                if not quiet: print("Finding levenshtein...", end="")
                startTime = time.time()
                results = search(prevStep[0], 1)
                if not quiet: print(" ({0:.2f})".format(time.time() - startTime))
                if len(results) < 10 and i < MIN_STEPS:
                    restart = True
                    print(choice, len(results))
                    break
                    
            done = False
            while not done:
                if choice == "anagram":
                    nextStep = random.choice(anagrams)
                else:
                    nextStep = random.choice(results)
    
                if nextStep[0] not in steps:
                    steps.append(nextStep[0])
                    done = True
    
            resultsCount = len(results) if choice == "levenshtein" else len(anagrams)
            if not quiet: print(nextStep, resultsCount, choice)
            prevStep = nextStep
    
        print("restart", restart)
        print()
    
    endWord = steps[-1]
    print(steps[1:-1])
    print("END WORD:", endWord)  
    return steps

t0 = time.time()
steps = generateChain(startWord=None, strict=True)
print("Elapsed {0:.2f}s".format(time.time() - t0))

startWord = steps[0]
endWord = steps[-1]

START WORD: AGUZZO
Finding levenshtein... (0.00)
levenshtein 4
restart True

START WORD: SOFFRO
Finding anagrams... (0.19)
anagram 7
restart True

START WORD: COGLIE
Finding levenshtein... (0.01)
('COOLIE', 1) 16 levenshtein
Finding levenshtein... (0.00)
levenshtein 6
restart True

START WORD: LEGAMI
Finding anagrams... (0.21)
('MELAGLI', 1) 49 anagram
Finding levenshtein... (0.01)
('BELAGLI', 1) 17 levenshtein
Finding levenshtein... (0.01)
('BELLAGLI', 1) 15 levenshtein
Finding anagrams... (0.87)
anagram 9
restart True

START WORD: DISC
Finding anagrams... (0.08)
('DISCS', 1) 11 anagram
Finding anagrams... (0.11)
anagram 1
restart True

START WORD: RIPESI
Finding anagrams... (0.20)
('RISPEDI', 1) 74 anagram
Finding anagrams... (0.50)
('DISEPARI', 1) 55 anagram
Finding anagrams... (1.55)
('PRESIDIAR', 1) 72 anagram
Finding anagrams... (1.58)
('ADERIRSI', 1) 27 anagram
Finding levenshtein... (0.01)
levenshtein 9
restart True

START WORD: SPEGNE
Finding levenshtein... (0.01)
levenshtein 

# checkValidChain

In [40]:
def isInDictionary(word):
    results = search(word, 0)
    return len(results) > 0

class WordStatus():
  UNCHECKED = 0
  VALID = 1
  INVALID = 2
  NOT_IN_DICT = 3

def checkValidChain(startWord, endWord, wordChain):
    prevWord = startWord
    isOk = True
    statuses = []
    print()
    for word in wordChain + [endWord]:
        inDictionary = isInDictionary(word)
        results = [t[0] for t in search(prevWord, 1)]
        anagrams = [t[0] for t in findAnagramsEx(filteredWords, prevWord, maxDiffLength=1, strict=False)]
        isWordOk = inDictionary and (word in results or word in anagrams) 
        if not inDictionary:
            print("✖️", word)
            statuses.append(WordStatus.NOT_IN_DICT)
        else:
            print("✔️" if isWordOk else "❌", word)
            statuses.append(WordStatus.VALID if isWordOk else WordStatus.INVALID)
        
        if not isWordOk:
            isOk = False
        prevWord = word

    return isOk, statuses
            
print("START WORD:", startWord)
print("END WORD  :", endWord)
checkValidChain(startWord, endWord, ["PAROLA"])

START WORD: SVISI
END WORD  : RIAMMISE

❌ PAROLA
❌ RIAMMISE


(False, [2, 2])

checkValidChain

In [20]:
def displayCheckValidChain(startWord, endWord, wordChain):
    print()
    print(startWord)
    ok, statuses = checkValidChain(startWord, endWord, wordChain)
    print(endWord)

    print()
    print("👍 WON" if ok else "👎 LOST")

    print()


In [21]:
# regenerate start/end words
t0 = time.time()
steps = generateChain(quiet=True, startWord='piccolo')
print("\nGenerate {0:.2f}s".format(time.time() - t0))

startWord = steps[0]
endWord = steps[-1]

print("START WORD:", startWord)
print("END WORD  :", endWord)

START WORD: VILLAN
restart False

END WORD: GELOSI

Generate 0.88s
START WORD: VILLAN
END WORD  : GELOSI


In [22]:
wordChain = [
    'pantera',
    'pentola',
    'pantofola',
    'osteria',
    'ristorai',
    'piantala',
]
wordChain = [w.strip().upper() for w in wordChain]
#print("GUESS WORD CHAIN", wordChain)

t0 = time.time()
displayCheckValidChain(startWord, endWord, wordChain)
print("Validate {0:.2f}s".format(time.time() - t0))


VILLAN
❌ PANTERA
❌ PENTOLA
❌ PANTOFOLA
❌ OSTERIA
❌ RISTORAI
❌ PIANTALA
❌ GELOSI
GELOSI

👎 LOST

Validate 3.92s


In [35]:
# STEPS

print("COMPUTED WORD CHAIN", steps)

COMPUTED WORD CHAIN ['VILLAN', 'LAVINO', 'SOLANI', 'LINOSA', 'LINOSO', 'GOLOSI', 'GELOSI']


In [24]:
# find in dictionary

findWord = 'muovi'

search(findWord.strip().upper(), 0)


[('MUOVI', 0)]

In [25]:
print([hex(ord(ch)) for ch in "✖️"])
print([hex(ord(ch)) for ch in "✔️"])
print([hex(ord(ch)) for ch in "❌"])
"👍"
hex(ord("👎"))

['0x2716', '0xfe0f']
['0x2714', '0xfe0f']
['0x274c']


'0x1f44e'

# server functions

In [26]:
@anvil.server.callable
def srvGenerateWordChain(quiet=True, startWord=None, strict=True):
  steps = generateChain(quiet=quiet, startWord=startWord, strict=strict)
  return steps

@anvil.server.callable
def srvValidateChain(startWord, endWord, wordChain):
  isOk, statuses = checkValidChain(startWord, endWord, wordChain)
  return isOk, statuses



# anagrams dict

In [27]:
t0 = time.time()

anagramsDict = {}
wordsByLength = {}
wordsBySortedAndLength = {}
for w in filteredWords[:]:
  sortedWord = ''.join(sorted(w))
  if sortedWord not in anagramsDict:
    anagramsDict[sortedWord] = set()
  anagramsDict[sortedWord].add(w)
  # moreAnagrams = findAnagramsEx(filteredWords, w, maxDiffLength=1, strict=True)
  # for item in moreAnagrams:
  #   anagramsDict[sortedWord].add(item[0])
  length = len(w)
  if length not in wordsByLength:
    wordsByLength[length] = set()
  wordsByLength[length].add(w)
  if w not in wordsBySortedAndLength:
    wordsBySortedAndLength[w] = [None, None, None]
  wordsBySortedAndLength[w][0] = sortedWord
  wordsBySortedAndLength[w][1] = length
  counter = Counter2(w)
  wordsBySortedAndLength[w][2] = counter


print(f'{(time.time() - t0):.2f}s')

display([item for item in random.choices(list(anagramsDict.items()), k=10)])
display([(k, list(v)[:5]) for k, v in wordsByLength.items()])
display([item for item in random.choices(list(wordsBySortedAndLength.items()), k=10)])


Reconnecting Anvil Uplink...
Connecting to wss://anvil.works/uplink
Anvil websocket open
Fatal error from Anvil server: Incorrect uplink key
Anvil websocket closed (code 1000, reason=b'')
4.21s


[('AABGIIMRV', {'IMBRAGAVI', 'IMBRIGAVA'}),
 ('CEHIIMNO', {'MINOICHE'}),
 ('AACINNRST', {'SCANNARTI', 'SCANTINAR', 'SCARNANTI', 'STRACANNI'}),
 ('EGGIIMMTU', {'MUGGITEMI'}),
 ('AEIIPPTV', {'APPETIVI', 'PEPATIVI', 'PIPATEVI'}),
 ('AABEEINSU', {'EUSEBIANA'}),
 ('AAEILQTUV', {'LIQUAVATE'}),
 ('AIMRZ', {'MARZI'}),
 ('CHIINOT', {'CHITINO', 'CHITONI'}),
 ('AAACHIPPT', {'CHIAPPATA'})]

[(9, ['RICAMARMI', 'DIFILARLE', 'SFIANCATA', 'MAZZICARE', 'TOMARTELO']),
 (6, ['BILOBE', 'NUOTAI', 'PLACCO', 'UGGERO', 'VIELLE']),
 (5, ['FERVO', 'SACRA', 'ATOUT', 'SMODA', 'ETINI']),
 (7, ['PISCOPI', 'FINTASI', 'MOZZALI', 'STESANO', 'FOLCERE']),
 (8, ['RASOIAVI', 'ODIRANNO', 'APPULCRI', 'PUNTAZZO', 'MALFATTE']),
 (4, ['BUCA', 'PIPE', 'VIGE', 'RENO', 'FICO'])]

[('MONTICALO',
  ['ACILMNOOT',
   9,
   {'M': 1, 'O': 2, 'N': 1, 'T': 1, 'I': 1, 'C': 1, 'A': 1, 'L': 1}]),
 ('SUOLALO', ['ALLOOSU', 7, {'S': 1, 'U': 1, 'O': 2, 'L': 2, 'A': 1}]),
 ('RELATIONS',
  ['AEILNORST',
   9,
   {'R': 1, 'E': 1, 'L': 1, 'A': 1, 'T': 1, 'I': 1, 'O': 1, 'N': 1, 'S': 1}]),
 ('SAGRARONO',
  ['AAGNOORRS', 9, {'S': 1, 'A': 2, 'G': 1, 'R': 2, 'O': 2, 'N': 1}]),
 ('GUSTOSITA',
  ['AGIOSSTTU', 9, {'G': 1, 'U': 1, 'S': 2, 'T': 2, 'O': 1, 'I': 1, 'A': 1}]),
 ('BRASILE',
  ['ABEILRS', 7, {'B': 1, 'R': 1, 'A': 1, 'S': 1, 'I': 1, 'L': 1, 'E': 1}]),
 ('BETTOLATA',
  ['AABELOTTT', 9, {'B': 1, 'E': 1, 'T': 3, 'O': 1, 'L': 1, 'A': 2}]),
 ('VEGANIANI',
  ['AAEGIINNV', 9, {'V': 1, 'E': 1, 'G': 1, 'A': 2, 'N': 2, 'I': 2}]),
 ('SMORTISCI',
  ['CIIMORSST', 9, {'S': 2, 'M': 1, 'O': 1, 'R': 1, 'T': 1, 'I': 2, 'C': 1}]),
 ('MALGAMELA', ['AAAEGLLMM', 9, {'M': 2, 'A': 3, 'L': 2, 'G': 1, 'E': 1}])]

In [28]:

# t0 = time.time()

# wordList = filteredWords[:21]
# for i, w in enumerate(wordList):
#   if i % 100 == 0:
#     print(f'{i:6}/{len(wordList)} {w}')
#   sortedWord, wordLength, sourceCounter = wordsBySortedAndLength[w]
  
#   candidates = []
#   targetLength = wordLength - 1
#   if targetLength in wordsByLength:
#     candidates = list(wordsByLength[targetLength])
#   targetLength = wordLength + 1
#   if targetLength in wordsByLength:
#     candidates += list(wordsByLength[targetLength])
  
#   for targetWord in candidates:
#     sortedTargetWord, targetLength, targetCounter = wordsBySortedAndLength[targetWord]
#     isAnagram, diff = _isAnagramEx2(sortedWord, sortedTargetWord, wordLength, targetLength, maxDiffLength=1, strict=True)
    
#     # if len(w) < len(targetWord):
#     #   sourceCounter, targetCounter = targetCounter, sourceCounter
#     # diffCounter = sourceCounter - targetCounter
#     # diffSum = sum(diffCounter.values())
#     # isAnagram = diffSum == 1

#     #sourceCounter.isAnagram(targetCounter, maxDiffLength = 1)

#     if isAnagram:
#       # print("->", targetWord)
#       anagramsDict[sortedWord].add(targetWord)
#       # display(anagramsDict[sortedWord])

# print(f'{(time.time() - t0):.2f}s')


In [29]:
word = 'ABAVI'
sortedWord, wordLength, sourceCounter = wordsBySortedAndLength[word]
display(anagramsDict[sortedWord])

_isAnagramEx2(list('BAVA'), list('ABAVI'), 4, 5, maxDiffLength=1, strict=True)


{'ABAVI', 'BIAVA'}

(False, 2)

In [30]:
print('LUCRAMELO'.__hash__())
print('LUCRAMELO'.__hash__())
print('lucramelo'.__hash__())

for k,v in wordsByLength.items():
  print(k, len(v))

#findAnagramsEx(filteredWords, 'TAVOLIERE', maxDiffLength=1, strict=True)

-6357037853119430282
-6357037853119430282
-8225445108557952560
9 200178
6 23116
5 9051
7 56496
8 109355
4 2927


# server wait

In [37]:
anvil.server.wait_forever()

START WORD: OSANTE
restart False

['OSTANTE', 'OSTANTI', 'COSTANTI', 'COSANTI', 'DOSANTI']
END WORD: DESINATO
START WORD: BAIULO
levenshtein 6
restart True

START WORD: INPUTO
levenshtein 7
restart True

START WORD: SPELTA
restart False

['LESTA', 'LASTA', 'SALATO', 'ALITOSA', 'ASSOLATI']
END WORD: ASSOLACI
START WORD: RODAMI
levenshtein 9
restart True

START WORD: ANTI
restart False

['CANTI', 'CACTI', 'CASTI', 'SCIATI', 'SCITALI']
END WORD: CITALI
START WORD: QUASSU
anagram 0
restart True

START WORD: RAREFA
levenshtein 4
restart True

START WORD: OPTAVO
levenshtein 7
restart True

START WORD: ERBAI
restart False

['RAIE', 'BAIE', 'BALIE', 'BALIA', 'ALBICA']
END WORD: CABLAMI
START WORD: SORATI
restart False

['SORBATI', 'BASIRO', 'BASTIRO', 'BASTIRLO', 'SABOTARLI']
END WORD: SABOTALI
START WORD: ARTATE
levenshtein 9
restart True

START WORD: RUPINA
restart False

['SUPINA', 'PINSA', 'PINNA', 'PENNA', 'SPENNA']
END WORD: SPENTA
START WORD: IBERA
levenshtein 4
restart True

START WORD

KeyboardInterrupt: ignored